In [5]:
from torch import nn

In [6]:
class Attention(nn.Module):
    def __init__(self, encoder_hidden_dim, decoder_hidden_dim):
        super().__init__()
 
        # The input dimension will the the concatenation of
        # encoder_hidden_dim (hidden) and  decoder_hidden_dim(encoder_outputs)
        self.attn_hidden_vector = nn.Linear(encoder_hidden_dim + decoder_hidden_dim, decoder_hidden_dim)
 
        # We need source len number of values for n batch as the dimension
        # of the attention weights. The attn_hidden_vector will have the
        # dimension of [source len, batch size, decoder hidden dim]
        # If we set the output dim of this Linear layer to 1 then the
        # effective output dimension will be [source len, batch size]
        self.attn_scoring_fn = nn.Linear(decoder_hidden_dim, 1, bias=False)
 
    def forward(self, hidden, encoder_outputs):
        # hidden = [1, batch size, decoder hidden dim]
        src_len = encoder_outputs.shape[0]
 
        # We need to calculate the attn_hidden for each source words.
        # Instead of repeating this using a loop, we can duplicate
        # hidden src_len number of times and perform the operations.
        hidden = hidden.repeat(src_len, 1, 1)
 
        # Calculate Attention Hidden values
        attn_hidden = torch.tanh(self.attn_hidden_vector(torch.cat((hidden, encoder_outputs), dim=2)))
 
        # Calculate the Scoring function. Remove 3rd dimension.
        attn_scoring_vector = self.attn_scoring_fn(attn_hidden).squeeze(2)
 
        # The attn_scoring_vector has dimension of [source len, batch size]
        # Since we need to calculate the softmax per record in the batch
        # we will switch the dimension to [batch size,source len]
        attn_scoring_vector = attn_scoring_vector.permute(1, 0)
 
        # Softmax function for normalizing the weights to
        # probability distribution
        return F.softmax(attn_scoring_vector, dim=1)
    
class Encoder(nn.Module):
    def __init__(self, vocab_len, embedding_dim, encoder_hidden_dim, n_layers=1, dropout_prob=0.5):
        super().__init__()
 
        self.embedding = nn.Embedding(vocab_len, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, encoder_hidden_dim, n_layers, dropout=dropout_prob)
 
        self.dropout = nn.Dropout(dropout_prob)
 
    def forward(self, input_batch):
        embedded = self.dropout(self.embedding(input_batch))
        outputs, hidden = self.rnn(embedded)
 
        return outputs, hidden

class OneStepDecoder(nn.Module):
    def __init__(self, input_output_dim, embedding_dim, encoder_hidden_dim, decoder_hidden_dim, attention, dropout_prob=0.5):
        super().__init__()
 
        self.output_dim = input_output_dim
        self.attention = attention
 
        self.embedding = nn.Embedding(input_output_dim, embedding_dim)
 
        # Add the encoder_hidden_dim and embedding_dim
        self.rnn = nn.GRU(encoder_hidden_dim + embedding_dim, decoder_hidden_dim)
        # Combine all the features for better prediction
        self.fc = nn.Linear(encoder_hidden_dim + decoder_hidden_dim + embedding_dim, input_output_dim)
 
        self.dropout = nn.Dropout(dropout_prob)
    
    def forward(self, input, hidden, encoder_outputs):
        # Add the source len dimension
        input = input.unsqueeze(0)
 
        embedded = self.dropout(self.embedding(input))
 
        # Calculate the attention weights
        a = self.attention(hidden, encoder_outputs).unsqueeze(1)
 
        # We need to perform the batch wise dot product.
        # Hence need to shift the batch dimension to the front.
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
 
        # Use PyTorch's bmm function to calculate the weight W.
        W = torch.bmm(a, encoder_outputs)
 
        # Revert the batch dimension.
        W = W.permute(1, 0, 2)
 
        # concatenate the previous output with W
        rnn_input = torch.cat((embedded, W), dim=2)
 
        output, hidden = self.rnn(rnn_input, hidden)
 
        # Remove the sentence length dimension and pass them to the Linear layer
        predicted_token = self.fc(torch.cat((output.squeeze(0), W.squeeze(0), embedded.squeeze(0)), dim=1))
 
        return predicted_token, hidden, a.squeeze(1)

class Decoder(nn.Module):
    def __init__(self, one_step_decoder, device):
        super().__init__()
        self.one_step_decoder = one_step_decoder
        self.device = device
 
    def forward(self, target, encoder_outputs, hidden, teacher_forcing_ratio=0.5):
        batch_size = target.shape[1]
        trg_len = target.shape[0]
        trg_vocab_size = self.one_step_decoder.output_dim
 
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        input = target[0, :]
 
        for t in range(1, trg_len):
            # Pass the encoder_outputs. For the first time step the 
            # hidden state comes from the encoder model.
            output, hidden, a = self.one_step_decoder(input, hidden, encoder_outputs)
            outputs[t] = output
 
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
 
            input = target[t] if teacher_force else top1
 
        return outputs

In [8]:
from seqgen.model import seq2seq_lstm as s

In [64]:
annotations = 2.0*torch.tensor([[[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4],[5,5,5,5]],[[2,2,2,2],[2,2,2,2],[2,2,2,2],[2,2,2,2],[2,2,2,2]]])
annotations, annotations.shape

(tensor([[[ 2.,  2.,  2.,  2.],
          [ 4.,  4.,  4.,  4.],
          [ 6.,  6.,  6.,  6.],
          [ 8.,  8.,  8.,  8.],
          [10., 10., 10., 10.]],
 
         [[ 4.,  4.,  4.,  4.],
          [ 4.,  4.,  4.,  4.],
          [ 4.,  4.,  4.,  4.],
          [ 4.,  4.,  4.,  4.],
          [ 4.,  4.,  4.,  4.]]]),
 torch.Size([2, 5, 4]))

In [65]:
hn = 1.0*torch.tensor([[1,2,3,4],[2,3,4,5]])
hn, hn.shape

(tensor([[1., 2., 3., 4.],
         [2., 3., 4., 5.]]),
 torch.Size([2, 4]))

In [66]:
hn_rep = s.repeat_hidden_state(hn, 5)
hn_rep, hn_rep.shape

(tensor([[[1., 2., 3., 4.],
          [1., 2., 3., 4.],
          [1., 2., 3., 4.],
          [1., 2., 3., 4.],
          [1., 2., 3., 4.]],
 
         [[2., 3., 4., 5.],
          [2., 3., 4., 5.],
          [2., 3., 4., 5.],
          [2., 3., 4., 5.],
          [2., 3., 4., 5.]]]),
 torch.Size([2, 5, 4]))

In [67]:
hn_rep.permute(0, 2, 1), annotations

(tensor([[[1., 1., 1., 1., 1.],
          [2., 2., 2., 2., 2.],
          [3., 3., 3., 3., 3.],
          [4., 4., 4., 4., 4.]],
 
         [[2., 2., 2., 2., 2.],
          [3., 3., 3., 3., 3.],
          [4., 4., 4., 4., 4.],
          [5., 5., 5., 5., 5.]]]),
 tensor([[[ 2.,  2.,  2.,  2.],
          [ 4.,  4.,  4.,  4.],
          [ 6.,  6.,  6.,  6.],
          [ 8.,  8.,  8.,  8.],
          [10., 10., 10., 10.]],
 
         [[ 4.,  4.,  4.,  4.],
          [ 4.,  4.,  4.,  4.],
          [ 4.,  4.,  4.,  4.],
          [ 4.,  4.,  4.,  4.],
          [ 4.,  4.,  4.,  4.]]]))

In [68]:
hn_st = torch.cat([hn_rep, annotations], dim=2)
hn_st, hn_st.shape

(tensor([[[ 1.,  2.,  3.,  4.,  2.,  2.,  2.,  2.],
          [ 1.,  2.,  3.,  4.,  4.,  4.,  4.,  4.],
          [ 1.,  2.,  3.,  4.,  6.,  6.,  6.,  6.],
          [ 1.,  2.,  3.,  4.,  8.,  8.,  8.,  8.],
          [ 1.,  2.,  3.,  4., 10., 10., 10., 10.]],
 
         [[ 2.,  3.,  4.,  5.,  4.,  4.,  4.,  4.],
          [ 2.,  3.,  4.,  5.,  4.,  4.,  4.,  4.],
          [ 2.,  3.,  4.,  5.,  4.,  4.,  4.,  4.],
          [ 2.,  3.,  4.,  5.,  4.,  4.,  4.,  4.],
          [ 2.,  3.,  4.,  5.,  4.,  4.,  4.,  4.]]]),
 torch.Size([2, 5, 8]))

In [69]:
lin = nn.Linear(8, 4)

In [70]:
lin(hn_st)

tensor([[[ 2.2170,  1.2324, -2.5259, -2.9899],
         [ 3.4794,  1.8973, -3.4077, -3.6192],
         [ 4.7417,  2.5621, -4.2896, -4.2486],
         [ 6.0041,  3.2270, -5.1714, -4.8779],
         [ 7.2665,  3.8918, -6.0532, -5.5072]],

        [[ 3.4373,  2.4055, -3.7759, -4.3160],
         [ 3.4373,  2.4055, -3.7759, -4.3160],
         [ 3.4373,  2.4055, -3.7759, -4.3160],
         [ 3.4373,  2.4055, -3.7759, -4.3160],
         [ 3.4373,  2.4055, -3.7759, -4.3160]]], grad_fn=<ViewBackward0>)

In [72]:
torch.bmm(hn_rep.permute(0,2,1), annotations)

tensor([[[ 30.,  30.,  30.,  30.],
         [ 60.,  60.,  60.,  60.],
         [ 90.,  90.,  90.,  90.],
         [120., 120., 120., 120.]],

        [[ 40.,  40.,  40.,  40.],
         [ 60.,  60.,  60.,  60.],
         [ 80.,  80.,  80.,  80.],
         [100., 100., 100., 100.]]])